> Contribution chính là đề xuất thuật toán để loại bỏ dịch chuyển, biến dạng ảnh không mong muốn của camera theo thời gian, đảm bảo các vị trí trong ảnh không bị thay đổi do các yếu tố bên ngoài (gió, nhiệt độ, ...)

# Data

* **Cao 43m**, cách đê chắn sóng 300m, cách bờ biển 100m về phía đất liền
* Tập trung camera **C1** bao phủ **75% bờ biển** dài **1.7km**
* Camera được quản lý bởi app **SIRENA** ghi hình ảnh theo 3 chế độ:
    * **Snapshot**: ảnh chụp tức thời
    * **Time exposure**: ảnh trung bình theo thời gian
    * **Standard deviation**: ảnh phương sai chuẩn
* Cứ mỗi **30 phút**, camera ghi lại ảnh trong **10 phút** với tốc độ **7.5Hz**

* Mực nước biển thay đổi theo mùa
* Có điều kiện nhật triều, biên độ mùa xuân là 75m
* Bãi biển **bồi tụ 6m/năm**


* Máy đo thủy triều **ADCP** đặt ở độ sâu 10m, cách bờ biển 10km

* Bãi biển kéo dài **2km** (với **20 cross-shore** transects with spacing of **100m**) 

# Method

## Environmental conditions

* Mức nước được tổng hợp lại bằng công thức:
$$ z_{total} = z_{gauge} + z_w + \frac{S}{2} $$

trong đó:
* $z_{gauge}$ là mực nước đo được từ máy đo **thủy triều**
* $z_w$ là mức nước dâng do **sóng** (wave setup):
$$ z_w = 0.016(H_0L_0)^{\frac{1}{2}} $$
* $S$ là **độ cao dao động** của nước do **sóng**:  
$$ S = 0.046(H_0L_0)^{\frac{1}{2}} $$

* Ngoài ra các yếu tố khí hậu: **gió mùa, không khí lạnh, bão** ảnh hưởng đáng kể đến sự thay đổi của đường bờ

## Quá trình xử lý hình ảnh

1️⃣**Image Stabilization Algorithm**

* ✍️ defining the **unstabilized image** to be corrected and **its closest stabilized image** in time from the **database** of monthly  reference images

* ✂️ Mỗi ảnh được tách thành **2 ROI** và được xử lý riêng biệt bởi **SURF**:
    * **ROI 1** ở gần, dễ phát hiện đặc trưng hơn
    * **ROI 2** ở xa, giảm ngưỡng threshold để cân bằng số đặc trưng

* 🔢 **SURF**:
    * `Detect features`: thường là góc, cạnh, điểm có độ tương phản cao, dựa vào bộ lọc **Haar wavelet** để tìm vùng có thay đổi mạnh về cường độ sáng
    * `Extract features`: tạo **feature descriptor** cho mỗi điểm đặc trưng
    * `Find mathching features`: tính khoảng cách **Euclide** giữa **descriptor** giữa ảnh cần ổn định và ảnh tham chiếu, nếu 2 điểm có khoảng cách đủ nhỏ, được coi là 1 cặp **matching feature** 

* 🔗 Concatanate **ROI 1** và **ROI 2**

* 🔍 Ước lượng phép biến đổi hình học (**geomtric transformation**) dựa trên **matching feature** đã tìm được ở trên

* 📝 Kiểm tra xem có **đủ features** trong phép biến đổi hình học cho mỗi **ROI** không? nếu không thì làm lại từ đầu

* Tích lũy **~ 20 lần** các phép biến đổi này để tránh nhiễu
* Nếu **số lượng** phép biến đổi ít hơn **5** thì **lower threshold** và **reset iterations**

* 📊 Sau cùng, **apply mean transformation**

⚠️ *Note: việc chạy lại vẫn giữ nguyên `ảnh tham chiếu`, chỉ chạy lại thuật toán `SURF` do tính ngẫu nhiên khi detect features*

# Trang 7